# Dickerson Lab NWB Reader

This notebook demonstrates how to read and explore NWB files created by the Dickerson Lab's conversion script.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pynwb import read_nwb
from pathlib import Path

## Load the NWB file

First, specify the path to your NWB file and load it using the NWBHDF5IO reader.

In [ ]:
# Replace with the path to your NWB file
nwb_file_path = Path("/home/heberto/cohen_project/Sample data/Dickerson Lab/nwb_files/Sample_2.nwb")

# Open the NWB file
nwbfile = read_nwb(nwb_file_path)
nwbfile

## Explore file metadata

Let's first look at the general metadata in the NWB file.

In [ ]:
# Display general file metadata
print(f"NWB File: {nwbfile.identifier}")
print(f"Session ID: {nwbfile.session_id}")
print(f"Session Description: {nwbfile.session_description}")
print(f"Session Start Time: {nwbfile.session_start_time}")

# Display subject information if available
if nwbfile.subject is not None:
    print("\nSubject Information:")
    print(f"Subject ID: {nwbfile.subject.subject_id}")
    if hasattr(nwbfile.subject, 'age'):
        print(f"Age: {nwbfile.subject.age}")
    if hasattr(nwbfile.subject, 'sex'):
        print(f"Sex: {nwbfile.subject.sex}")
    if hasattr(nwbfile.subject, 'genotype'):
        print(f"Genotype: {nwbfile.subject.genotype}")

## Explore devices and imaging metadata

The Dickerson Lab NWB file contains metadata about the Thor imaging system and other devices.

In [ ]:
# Display devices
print("Devices:")
for device_name, device in nwbfile.devices.items():
    print(f"- {device_name}")
    if hasattr(device, 'description') and device.description is not None:
        print(f"  Description: {device.description}")
    if hasattr(device, 'manufacturer') and device.manufacturer is not None:
        print(f"  Manufacturer: {device.manufacturer}")

In [ ]:
# Display imaging plane information
if hasattr(nwbfile, 'imaging_planes') and len(nwbfile.imaging_planes) > 0:
    print("\nImaging Planes:")
    for plane_name, plane in nwbfile.imaging_planes.items():
        print(f"- {plane_name}")
        print(f"  Description: {plane.description}")
        print(f"  Indicator: {plane.indicator}")
        print(f"  Location: {plane.location}")
        print(f"  Excitation Lambda: {plane.excitation_lambda} nm")
        
        # Display optical channel information
        for i, channel in enumerate(plane.optical_channel):
            print(f"  Optical Channel {i+1}: {channel.name}")
            print(f"    Description: {channel.description}")
            if hasattr(channel, 'emission_lambda') and channel.emission_lambda is not None:
                print(f"    Emission Lambda: {channel.emission_lambda} nm")

## Explore two-photon imaging data

The Dickerson Lab NWB file contains two-photon calcium imaging data with GCaMP and tdTomato indicators.

In [ ]:
# Check for two-photon series data
if hasattr(nwbfile, 'acquisition') and len(nwbfile.acquisition) > 0:
    two_photon_series = [name for name in nwbfile.acquisition if 'two_photon' in name.lower() or 'twophoton' in name.lower()]
    
    if two_photon_series:
        print("Two-Photon Series in acquisition:")
        for name in two_photon_series:
            data = nwbfile.acquisition[name]
            print(f"- {name}")
            print(f"  Description: {data.description}")
            print(f"  Data shape: {data.data.shape}")
            print(f"  Imaging plane: {data.imaging_plane.name}")
            print(f"  Rate: {data.rate} Hz")
            print(f"  Unit: {data.unit}")
    else:
        print("No two-photon series found in acquisition.")

## Explore behavioral data

The Dickerson Lab NWB file contains behavioral data including wing kinematics and visual stimulus tracking.

In [ ]:
# List all acquisition data
print("Available acquisition data:")
for name in nwbfile.acquisition:
    if 'two_photon' not in name.lower() and 'twophoton' not in name.lower():
        print(f"- {name}")

In [ ]:
# Plot wing kinematics data (if available)
wing_data_names = [name for name in nwbfile.acquisition if 'wing' in name.lower()]

if wing_data_names:
    plt.figure(figsize=(12, 8))
    for i, name in enumerate(wing_data_names):
        data = nwbfile.acquisition[name]
        plt.subplot(len(wing_data_names), 1, i+1)
        
        # Get timestamps and data
        timestamps = data.timestamps[:] if data.timestamps is not None else np.arange(len(data.data))
        plt.plot(timestamps, data.data[:])
        
        plt.title(name)
        plt.xlabel('Time (s)')
        plt.ylabel(f'{data.unit}')
    
    plt.tight_layout()
    plt.show()
else:
    print("No wing kinematics data found in the file.")

## Explore visual stimulus data

The Dickerson Lab NWB file contains visual stimulus data tracking in X and Y directions.

In [ ]:
# Check for stimulus data
stimulus_data_names = [name for name in nwbfile.stimulus if 'visual' in name.lower() or 'stimulus' in name.lower()]

if stimulus_data_names:
    plt.figure(figsize=(12, 6))
    for i, name in enumerate(stimulus_data_names):
        data = nwbfile.stimulus[name]
        
        # Get timestamps and data
        timestamps = data.timestamps[:] if data.timestamps is not None else np.arange(len(data.data))
        
        # Check if data is 2D (X and Y coordinates)
        if len(data.data.shape) > 1 and data.data.shape[1] == 2:
            plt.subplot(2, 1, 1)
            plt.plot(timestamps, data.data[:, 0], label=f"{name} - X")
            plt.title(f"{name} - X Coordinate")
            plt.xlabel('Time (s)')
            plt.ylabel(f'{data.unit}')
            
            plt.subplot(2, 1, 2)
            plt.plot(timestamps, data.data[:, 1], label=f"{name} - Y")
            plt.title(f"{name} - Y Coordinate")
            plt.xlabel('Time (s)')
            plt.ylabel(f'{data.unit}')
        else:
            plt.plot(timestamps, data.data[:], label=name)
            plt.title(name)
            plt.xlabel('Time (s)')
            plt.ylabel(f'{data.unit}')
    
    plt.tight_layout()
    plt.show()
else:
    print("No visual stimulus data found in the file.")

## Visualize two-photon imaging data

Let's visualize a frame from the two-photon imaging data if available.

In [ ]:
# Find two-photon series data
two_photon_series = [name for name in nwbfile.acquisition if 'two_photon' in name.lower() or 'twophoton' in name.lower()]

if two_photon_series:
    for name in two_photon_series:
        data = nwbfile.acquisition[name]
        
        # Get a single frame (the first one)
        frame_index = 0
        frame = data.data[frame_index]
        
        plt.figure(figsize=(10, 8))
        plt.imshow(frame, cmap='gray')
        plt.title(f"{name} - Frame {frame_index}")
        plt.colorbar(label='Intensity')
        plt.show()
        
        print(f"Displayed frame {frame_index} from {name}")
        print(f"Frame shape: {frame.shape}")
else:
    print("No two-photon series data found to visualize.")